# Who's the biggest tax evader?

#### Imports:

In [1]:
import plotly.plotly as py
import pandas as pd
import pycountry
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

## 1. Data cleaning and preprocessing

In [2]:
# Load country codes
df_country_codes = pd.read_csv('data/countries_codes.csv', low_memory=False).set_index('COUNTRY')

In [3]:
# Load datasets
## Load panama papers datasets
pp_edges = pd.read_csv('data/panama_papers/panama_papers.edges.csv', low_memory=False)
pp_nodes_address = pd.read_csv('data/panama_papers/panama_papers.nodes.address.csv', low_memory=False)
pp_nodes_entity = pd.read_csv('data/panama_papers/panama_papers.nodes.entity.csv', low_memory=False)
pp_nodes_intermediary = pd.read_csv('data/panama_papers/panama_papers.nodes.intermediary.csv', low_memory=False)
pp_nodes_officer = pd.read_csv('data/panama_papers/panama_papers.nodes.officer.csv', low_memory=False)
## Load UN datasets
un_hdi_components_2014 = pd.read_csv('data/un/hdi_components.csv', low_memory=False)
un_gdp_per_capita = pd.read_csv('data/un/gdp_per_capita.csv', low_memory=False)
un_gdp_per_capita_ppp = pd.read_csv('data/un/gdp_per_capita_PPP.csv', low_memory=False)
## Load world bank datasets
wb_gini = pd.read_csv('data/world_bank/gini_index.csv', low_memory=False)
wb_income_share_20_per = pd.read_csv('data/world_bank/income_share_20_per.csv', low_memory=False)
wb_population_total = pd.read_csv('data/world_bank/population_total.csv', low_memory=False)

In [4]:
# TIM
wb_co2 = pd.read_excel('data/co2_emissions.xls')

In [5]:
# We only consider statistics that date from 2000 onwards
years_to_drop = list(map(str, np.arange(1960, 2000)))
wb_gini = wb_gini.drop(columns=years_to_drop)
wb_income_share_20_per = wb_income_share_20_per.drop(columns=years_to_drop)

In [6]:
# We select the rightmost value (most recent) for each row
gini_values = wb_gini.stack().groupby(level=0).last().reindex(wb_gini.index)

# Only select valid values and label other values as NaN
wb_gini['Gini'] = pd.to_numeric(gini_values, errors='coerce')

# Only select relevant columns
wb_gini = wb_gini[['Country Name', 'Country Code', 'Gini']]

In [7]:
# We select the rightmost value (most recent) for each row
income_share_20_per_values = wb_income_share_20_per.stack().groupby(level=0).last().reindex(wb_income_share_20_per.index)

# Only select valid values and label other values as NaN
wb_income_share_20_per['Income Share'] = pd.to_numeric(income_share_20_per_values, errors='coerce')

# Only select relevant columns
wb_income_share_20_per = wb_income_share_20_per[['Country Name', 'Country Code', 'Income Share']]

In [8]:
# TIM

# We select the rightmost value (most recent) for each row
wb_co2_values = wb_co2.stack().groupby(level=0).last().reindex(wb_co2.index)

# Only select valid values and label other values as NaN
wb_co2['CO2 Emissions'] = pd.to_numeric(wb_co2_values, errors='coerce')

# Only keep most recent values for each country
wb_co2 = wb_co2[['Country Name', 'Country Code', 'CO2 Emissions']]

# Remove countries without indicator information
wb_co2 = wb_co2.dropna()

In [9]:
# Join UN datasets with country codes DataFrame
un_hdi_components_2014 = un_hdi_components_2014.join(df_country_codes, on='Country')
un_gdp_per_capita = un_gdp_per_capita.join(df_country_codes, on='Country')
un_gdp_per_capita_ppp = un_gdp_per_capita_ppp.join(df_country_codes, on='Country')

In [10]:
# List of UN DataFrames
un_dfs = [un_hdi_components_2014, un_gdp_per_capita, un_gdp_per_capita_ppp]

# Define dictionary containing pairs (country name: ISO country code)
countries = dict()

for country in pycountry.countries:
    countries[country.name] = country.alpha_3  

for df in un_dfs:
    nan_values = df['CODE'].isna()
    input_countries = list(df[nan_values]['Country'].values)
        
    codes = []
    for country in input_countries:
        if country in countries:
            codes.append(countries.get(country))
        else:        
            accepted = []
            str_country = str(country)
            # check if string contains either common or official country name
            for p_country in pycountry.countries:
                if p_country.name in str_country or (hasattr(p_country, 'common_name') and p_country.common_name in str_country):
                    accepted.append(p_country.alpha_3)
            if len(accepted) == 1:
                codes.append(accepted[0])
            else:
                codes.append(None)

    df.loc[nan_values, 'CODE'] = codes
    # Remove rows that were not found
    df = df[df['CODE'].notnull()]

In [11]:
pp_references_country = pp_nodes_address.groupby(['country_codes', 'countries']).size().reset_index(name='counts')

In [12]:
wb_population_2014 = wb_population_total[['Country Code', '2014']]
occurrence_pop = pp_references_country.merge(wb_population_2014, left_on='country_codes', right_on='Country Code')
occurrence_pop['counts_1000'] = 1000 * occurrence_pop['counts'] / occurrence_pop['2014']

## 2. Data analysis and observations

### 2.1 Panama Papers and population

In [13]:
pp_intermediary_country = pp_nodes_intermediary.groupby(['country_codes', 'countries']).size().reset_index(name='counts')
pp_intermediary_country = pp_intermediary_country.sort_values('counts', ascending=False)

We display the distribution using a map:

In [14]:
data = [ dict(
        type = 'choropleth',
        locations = pp_intermediary_country['country_codes'],
        z = pp_intermediary_country['counts'],
        text = pp_intermediary_country['countries'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Number of references'),
      ) ]

"""
layout = {
  "geo": {
    "coastlinewidth": 2, 
    "countrycolor": "rgb(204, 204, 204)", 
    "lakecolor": "rgb(255, 255, 255)", 
    "landcolor": "rgb(204, 204, 204)", 
    "lataxis": {
      "dtick": 10, 
      "range": [20, 60], 
      "showgrid": True
    }, 
    "lonaxis": {
      "dtick": 20, 
      "range": [-100, 20], 
      "showgrid": True
    }, 
    "projection": {"type": "equirectangular"}, 
    "resolution": 50, 
    "showlakes": True, 
    "showland": False
  }, 
  "showlegend": False, 
  "title": "Seoul to Hong Kong Great Circle"
}

"""
layout = dict(
    title = 'References in Panama Papers',
    geo = dict(
        showcountries = True,
        countrycolor = "rgb(217, 217, 217)",
        showframe = False,
        resolution=50,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        ),
        bgcolor = 'rgba(255, 255, 255, 0.0)',
    )
)


fig = dict( data=data, layout=layout )

iplot( fig, validate=False)

In [15]:
min_count = pp_intermediary_country['counts'].min()
max_count = pp_intermediary_country['counts'].max()

In [16]:
def firstOrDefault(values, default):
    if values is None or len(values) == 0:
        return default
    return values[0]

## 3. Milestone 3

We can see from the tables that most of the countries involved in the Panama Papers affair are small islands, which unfortunately are not displayed by the `Plotly` library. For the next milestone, we will fix that issue either by finding a solution that still works with `Plotly` or by using a different library, such as `folium`.

So far, we have made insightful observations that match the reports found in the media, particularly about which countries were most involved in this affair.

For the next milestone, we will further investigate the links between the countries, and try to understand the correlation of socio-economic factors with the locations of entities, officers and intermediaries involved in Panama Papers. More specifically, we intend to:
- Find which socio-economic factors are correlated with the results we found so far, and how they are correlated
- Display the links between the countries using a graph similar to the one found [here](https://plot.ly/python/lines-on-maps/)
- Fix issues with certain countries (particularly small islands) not being displayed in the graph
-

In [17]:
# only keep columns relevant for plotting data
nodes_intermediary_parsed = pp_nodes_intermediary[['node_id', 'country_codes', 'countries']]
nodes_entity_parsed = pp_nodes_entity[['node_id', 'country_codes', 'countries']]
nodes_officer_parsed = pp_nodes_officer[['node_id', 'country_codes', 'countries']]
nodes_address_parsed = pp_nodes_address[['node_id', 'country_codes', 'countries']]

## National and international links: geographical representation

Since we will be displaying the different nodes on a world map, we need the latitude and longitude of each country. We use the following [dataset](https://opendata.socrata.com/dataset/Country-List-ISO-3166-Codes-Latitude-Longitude/mnkm-8ram) provided by Socrata to get the average latitude and longitude of every country. We store the dataset as a DataFrame and clean it:

In [18]:
countries_location = pd.read_csv('data/countries_latitude_longitude.csv')
countries_location = countries_location[['Alpha-3 code', 'Latitude (average)', 'Longitude (average)']]
countries_location = countries_location.rename(columns={'Alpha-3 code': 'Code', 'Latitude (average)': 'lat', 
                                                        'Longitude (average)': 'long'})

We are interested in the nodes in Panama Papers that have a country attached to them, so that we can see how different people and firms and companies interact on an international level. We create a DataFrame `nodes` to represent the node ID and country representing that node, as well as the country's central coordinates that we previously obtained.

In [19]:
# DataFrame containing (id, country code, country name) for every node in Panama Papers
nodes = nodes_entity_parsed.append(nodes_intermediary_parsed).append(nodes_officer_parsed).append(nodes_address_parsed)

# certain nodes are irrepresentative, without a country code and name, so we drop them
nodes = nodes.dropna()

# set latitude and longitude for every node
nodes = nodes.merge(countries_location, left_on='country_codes', right_on='Code').drop(columns='Code')

We define a DataFrame containing the geographical location of origin and destination of each edge in Panama Papers:

In [20]:
# get country of origin nodes
edges_countries = nodes.merge(pp_edges[['START_ID', 'TYPE', 'END_ID']], left_on='node_id', right_on='START_ID')
edges_countries = edges_countries.rename(columns={'node_id': 'id_1', 'country_codes': 'cc_1', 
                                                        'countries': 'country_1', 'lat': 'lat_1', 'long': 'long_1'})

# get country of destination nodes
edges_countries = edges_countries.merge(nodes, left_on='END_ID', right_on='node_id')
edges_countries = edges_countries.rename(columns={'node_id': 'id_2', 'country_codes': 'cc_2', 
                                                        'countries': 'country_2', 'lat': 'lat_2', 'long': 'long_2'})

# only keep relevant columns
edges_countries = edges_countries.drop(columns=['id_1', 'id_2'])

We can now separate the DataFrame containing edges into two DataFrames, one containing edges that are within a country and one containing edges that are international:

In [47]:
# mask to get edges that are within a country
within_country = edges_countries['cc_1'] == edges_countries['cc_2']

# remove countries that do not have international connections
edges_international = edges_countries[~within_country]
edges_national = edges_countries[within_country]

print("Number of international edges %d" % len(edges_international))
print("Number of national edges: %d" % len(edges_national))
print("Ratio of international to national edges: %.2f" % (len(edges_international)/len(edges_national)))

# get total number of national edges by country
edges_national = edges_national.groupby(['cc_1', 'country_1']).size().reset_index(name='count')

# get total number of international edges by each pair of countries
# reset index twice for future data manipulation
edges_international = edges_international.groupby(['cc_1', 'country_1', 'cc_2', 'country_2', 
                                'long_1', 'lat_1', 'long_2', 'lat_2']).size().reset_index(name='count')\
                                .sort_values('count', ascending=False).reset_index(drop=True)

Number of international edges 150439
Number of national edges: 381274
Ratio of international to national edges: 0.39


We see that the majority of edges are national, with approximately twice as many national edges when compared to international ones.

### National links

We display a world map showing the distribution of national links in the Panama Papers in different countries:

In [22]:
data = [ dict(
        type = 'choropleth',
        locations = edges_national['cc_1'],
        z = edges_national['count'],
        text = edges_national['country_1'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '',
            title = 'Number of references'),
      ) ]

layout = dict(
    title = 'Distribution of national Panama Papers links',
    geo = dict(
        showcountries = True,
        countrycolor = "rgb(217, 217, 217)",
        showframe = False,
        resolution=10,
        showcoastlines = False,
        projection = dict(type = 'Mercator'),
        bgcolor = 'rgba(255, 255, 255, 0.0)',
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, validate=False)

We can see from the world map that Switzerland has almost one sixth of all the national links, and that China has an additional eighth of the links. This is unsurprising, as Switzerland's economy is strongly focused on banks, and China has the world's largest population for a country.

### International links

We will now consider international links in Panama Papers. We will be representing these links as an undirected graph, so we must add links that have origin country A and destination country B and add them to links that have origin country B and destination country A. The nodes of our graph are the countries involved in Panama Papers, and we store these nodes in `nodes_countries`. We construct the international links graph in the following way:

In [23]:
# nodes are countries, so former node_id values are irrelevant
nodes_countries = nodes.drop(columns='node_id').drop_duplicates()

In [26]:
# we reverse the direction of the edges
edges_international_rev = edges_international.rename(columns={'cc_1': 'cc_2', 'cc_2': 'cc_1',
                                'country_1': 'country_2', 'country_2': 'country_1',
                                'long_1': 'long_2', 'long_2': 'long_1',
                                'lat_1': 'lat_2', 'lat_2': 'lat_1'})

# add edges in original direction with those in reverse direction
edges_international_total = edges_international.append(edges_international_2, sort=True).groupby(['cc_1', 'cc_2', 
                                    'country_1', 'country_2', 'lat_1', 'lat_2', 'long_1', 'long_2']).sum().reset_index()

# filter edges by lexographic order, to remove any duplicate edges
edges_international_total = edges_international_total[edges_international_total.apply(
                                lambda r: r['cc_1'] < r['cc_2'] , axis=1)]

In [54]:
countries = [ dict(
        type = 'scattergeo',
        lon = nodes_countries['long'],
        lat = nodes_countries['lat'],
        hoverinfo = 'text',
        text = nodes_countries['countries'],
        mode = 'markers',
        marker = dict( 
            size=3, 
            color='rgb(0, 0, 0)',
            line = dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        ))]

edges = []
for i in range(len(edges_international_total)):    
    edges.append(
        dict(type = 'scattergeo',
            lon = [edges_international_total['long_1'].iloc[i], edges_international_total['long_2'].iloc[i]],
            lat = [edges_international_total['lat_1'].iloc[i], edges_international_total['lat_2'].iloc[i]],
            mode = 'lines',
            hoverinfo = 'none',
            line = dict(
                width =  max(edges_international_total['count'].iloc[i]/2000, .02),
                color = 'red'
            ))
    )
    
layout = dict(
        title = 'International Panama Papers links',
        showlegend = False, 
        geo = dict(
        showcountries = True,
        countrycolor = "rgb(217, 217, 217)",
        showframe = False,
        resolution = 10,
        showcoastlines = False,
        projection = dict(type = 'Mercator'),
        bgcolor = 'rgba(255, 255, 255, 0.0)'
    ))
    
fig = dict(data=edges + countries, layout=layout)
iplot(fig, validate=False)

From the geographic representation of the distribution of Panama Papers links above, we notice that the majority of links are transatlantic. Specifically, most of the large links are between Central America and Central Europe. However, the largest link is between China and Hong Kong. This is logical, because there are over 20000 links originating from China to Hong Kong, which is by far the largest number of connections between any two countries (the next largest one is approximately 5000). We can also see some other "link hubs" that we expected to see, like the UAE, the Bahamas and Singapore. The UAE and Singapore are both big business centers with many international links, and the Bahamas is one of the many visible fiscal paradises on the map, because there is no tax on personal income or capital gains for citizens and residents.

## International links: network representation

In [31]:
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, BoxZoomTool,\
        ResetTool, LassoSelectTool, WheelZoomTool, PanTool
from bokeh.palettes import Spectral4, Inferno, Viridis
from bokeh.transform import linear_cmap
output_notebook()

Loading BokehJS ...

In [32]:
nodes_degrees = pd.DataFrame(edges_international_total.groupby('country_1').size()).reset_index().rename(columns={0: 'degree'})
nodes_degrees['country'] = nodes_degrees['country_1']

edges_network = edges_international_total[['country_1', 'country_2', 'count']]
edges_network = edges_network.merge(nodes_degrees)

# only destination countries
additional_nodes = list(set(edges_international_total['country_2']).difference(set(edges_international_total['country_1'])))

additional_nodes_names_dict = {}
for node in additional_nodes:
    additional_nodes_names_dict[node] = node 

nodes_names_dict = nodes_degrees.set_index('country_1')['country'].to_dict()
nodes_names_dict.update(additional_nodes_names_dict)

In [33]:
G = nx.from_pandas_edgelist(edges_network, source='country_1', target='country_2')
G.add_nodes_from(additional_nodes)
nx.set_node_attributes(G, nodes_names_dict, name='country')

In [34]:
plot = Plot(plot_width=600, plot_height=600, x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Panama Papers connections"

node_hover_tool = HoverTool(tooltips=[("Country", "@country"),]) # ("Degree", "@degree")])

plot.add_tools(node_hover_tool, TapTool(), BoxSelectTool(), BoxZoomTool(), ResetTool(), 
               LassoSelectTool(), WheelZoomTool(), PanTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)

# output_file("networkx_graph.html")
show(plot)

#### Network distribution with varying edge opacity

In [35]:
nodes_degrees = pd.DataFrame(edges_international_total.groupby('country_1').size()).reset_index().rename(columns={0: 'degree'})
nodes_degrees['country'] = nodes_degrees['country_1']

edges_network = edges_international_total[['country_1', 'country_2', 'count']]
edges_network = edges_network.merge(nodes_degrees)

# only destination countries
additional_nodes = list(set(edges_international_total['country_2']).difference(set(edges_international_total['country_1'])))
# additional_nodes_degrees_dict = {}
additional_nodes_names_dict = {}
for node in additional_nodes:
    additional_nodes_names_dict[node] = node 

nodes_names_dict = nodes_degrees.set_index('country_1')['country'].to_dict()
nodes_names_dict.update(additional_nodes_names_dict)

In [36]:
G = nx.from_pandas_edgelist(edges_network, source='country_1', target='country_2')
G.add_nodes_from(additional_nodes)
nx.set_node_attributes(G, nodes_names_dict, name='country')

In [37]:
edge_attrs = {}
for origin, dest in G.edges():
    query = edges_network.loc[(edges_network['country_1'] == origin)].loc[(edges_network['country_2'] == dest)]
    if len(query) > 0:
        count = query.iloc[0]['count']
        edge_attrs[(origin, dest)] = min(max(count / 2000, 0.1), 1)

In [38]:
nx.set_edge_attributes(G, edge_attrs, "edge_opacity")

In [39]:
plot = Plot(plot_width=600, plot_height=600, x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Panama Papers connections"

node_hover_tool = HoverTool(tooltips=[("Country", "@country"),]) # ("Degree", "@degree")])

plot.add_tools(node_hover_tool, TapTool(), BoxSelectTool(), BoxZoomTool(), ResetTool(), 
               LassoSelectTool(), WheelZoomTool(), PanTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=10, fill_color=Inferno[4][2], line_color="#000000")
graph_renderer.node_renderer.selection_glyph = Circle(size=12, fill_color=Inferno[6][4])
graph_renderer.node_renderer.hover_glyph = Circle(size=12, fill_color=Inferno[5][3])

graph_renderer.edge_renderer.glyph = MultiLine(line_color=Inferno[5][1], line_alpha="edge_opacity", line_width=2)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Inferno[6][2], line_width=3)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Inferno[7][3], line_width=2)

# for colors https://bokeh.pydata.org/en/latest/docs/reference/palettes.html

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)

# output_file("networkx_graph.html")
show(plot)

#### Network distribution with node centrality

In [40]:
nodes_degrees = pd.DataFrame(edges_international_total.groupby('country_1').size()).reset_index().rename(columns={0: 'degree'})
nodes_degrees['country'] = nodes_degrees['country_1']

edges_network = edges_international_total[['country_1', 'country_2', 'count']]
edges_network = edges_network.merge(nodes_degrees)

# only destination countries
additional_nodes = list(set(edges_international_total['country_2']).difference(set(edges_international_total['country_1'])))
additional_nodes_names_dict = {}
for node in additional_nodes:
    additional_nodes_names_dict[node] = node 

nodes_names_dict = nodes_degrees.set_index('country_1')['country'].to_dict()
nodes_names_dict.update(additional_nodes_names_dict)

In [41]:
edge_attrs = {}
for origin, dest in G.edges():
    query = edges_network.loc[(edges_network['country_1'] == origin)].loc[(edges_network['country_2'] == dest)]
    if len(query) > 0:
        count = query.iloc[0]['count']
        edge_attrs[(origin, dest)] = min(max(count / 2000, 0.1), 1)

In [42]:
nodes_centrality_dict = nx.betweenness_centrality(G)
for k, v in nodes_centrality_dict.items():
    nodes_centrality_dict[k] = np.clip(int(v*200), a_min=3, a_max=40)

In [43]:
G = nx.from_pandas_edgelist(edges_network, source='country_1', target='country_2')
G.add_nodes_from(additional_nodes)
nx.set_node_attributes(G, nodes_names_dict, name='country')
nx.set_edge_attributes(G, edge_attrs, name='edge_opacity')
nx.set_node_attributes(G, nodes_centrality_dict, name='centrality')

In [44]:
plot = Plot(plot_width=600, plot_height=600, x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Panama Papers connections"

node_hover_tool = HoverTool(tooltips=[("Country", "@country")])

plot.add_tools(node_hover_tool, TapTool(), BoxSelectTool(), BoxZoomTool(), ResetTool(), 
               LassoSelectTool(), WheelZoomTool(), PanTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size="centrality", fill_color=Inferno[4][2], line_color="#000000")
graph_renderer.node_renderer.selection_glyph = Circle(size=12, fill_color=Inferno[6][4])
graph_renderer.node_renderer.hover_glyph = Circle(size=12, fill_color=Inferno[5][3])

graph_renderer.edge_renderer.glyph = MultiLine(line_color=Inferno[5][1], line_alpha="edge_opacity", line_width=2)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Inferno[6][2], line_width=3)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Inferno[7][3], line_width=2)

# for colors https://bokeh.pydata.org/en/latest/docs/reference/palettes.html

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph_renderer)

# output_file("networkx_graph.html")
show(plot)

#### Case study: Queen of England

In [45]:
# http://www.asiaone.com/world/what-are-panama-papers-and-who-have-been-implicated

# daughter of former chinese premier: li xiaolin
li_xiaolin = pp_nodes_entity[pp_nodes_entity['name'].str.contains('COFIC INVESTMENTS LTD.')]

# president of argentina: mauricio macri
mauricio_macri = pp_nodes_entity[pp_nodes_entity['name'].str.contains('FLEG TRADING LTD')]

# football player: lionel messi
lionel_messi = pp_nodes_entity[pp_nodes_entity['name'].str.contains('MEGA STAR ENTERPRISES')]

# uae abu dhabi emir: khalifa bin zayed
khalifa_bin_zayed = pp_nodes_officer[pp_nodes_officer['name'].str.contains('KHALIFA BIN ZAYED')]

ValueError: cannot index with vector containing NA / NaN values

In [ ]:
pp_nodes_officer = pp_nodes_officer.dropna(subset=['name'])

In [ ]:
pp_nodes_entity = pp_nodes_entity.dropna(subset=['name'])

In [ ]:
pp_nodes_address[pp_nodes_address['address'].str.contains('Bermuda')]

In [ ]:
pp_nodes_entity[pp_nodes_entity['name'].str.contains('WL ROSS')]

In [ ]:
pp_nodes_officer[pp_nodes_officer['name'].str.contains('Ellen Johnson')]